In [1]:
import numpy as np
import pandas as pd
from os import listdir
import random
random.seed()   
import sklearn.mixture as mix
# import talib
import matplotlib.pyplot as plt
from matplotlib import cm
import time
%matplotlib inline
import seaborn as sns
from scipy.stats import pearsonr
from hmmlearn import hmm
np.random.seed(42)

ImportError: cannot import name '_hmmc'

# Load data from Reuters

In [ ]:
import eikon as ek
ek.set_app_id('604ea22425e048d39af0ef760ec9c64ebbe1fe68')
startdate='1000-01-20T15:04:05'
current_date=time.strftime("%Y-%m-%d")

TTF=ek.get_timeseries('TFMBYc1',start_date=startdate,end_date=current_date)
TTF = TTF.rename(columns={'CLOSE':'TTF_CLOSE'})
TTF = TTF.dropna(subset=['TTF_CLOSE'])
TTF['TTF_RSI'] = talib.RSI(TTF['TTF_CLOSE'],14).astype(float)
# display(TTF)

brent=ek.get_timeseries('LCOc12',start_date=startdate,end_date=current_date) 
brent = brent.rename(columns={'CLOSE':'brent_CLOSE'})
brent = brent.dropna(subset=['brent_CLOSE'])
brent['brent_RSI'] = talib.RSI(brent['brent_CLOSE'],14).astype(float)
# display(brent)

coal=ek.get_timeseries('TRAPI2Yc1',start_date=startdate,end_date=current_date)
coal = coal.rename(columns={'CLOSE':'coal_CLOSE'})
coal = coal.dropna(subset=['coal_CLOSE'])
# display(coal)
coal['coal_RSI'] = talib.RSI(coal['coal_CLOSE'],14).astype(float)
# display(coal)

CO2=ek.get_timeseries('CFI2c12',start_date=startdate,end_date=current_date)
CO2 = CO2.rename(columns={'CLOSE':'CO2_CLOSE'})
CO2 = CO2.dropna(subset=['CO2_CLOSE'])
CO2['CO2_RSI'] = talib.RSI(CO2['CO2_CLOSE'],14).astype(float)
# display(CO2)

data = pd.merge(pd.DataFrame(TTF['TTF_RSI']), pd.DataFrame(brent['brent_RSI']), how = 'inner', left_index=True, right_index=True)
data = pd.merge(data, pd.DataFrame(coal['coal_RSI']), how = 'inner', left_index=True, right_index=True)
data = pd.merge(data, pd.DataFrame(CO2['CO2_RSI']), how = 'inner', left_index=True, right_index=True)

data = pd.merge(data, pd.DataFrame(TTF['TTF_CLOSE']), how = 'inner', left_index=True, right_index=True)
data = pd.merge(data, pd.DataFrame(brent['brent_CLOSE']), how = 'inner', left_index=True, right_index=True)
data = pd.merge(data, pd.DataFrame(coal['coal_CLOSE']), how = 'inner', left_index=True, right_index=True)
data = pd.merge(data, pd.DataFrame(CO2['CO2_CLOSE']), how = 'inner', left_index=True, right_index=True)

select = data.ix[:].dropna()
data = data.dropna()

data['brent_Corr'] = data['TTF_CLOSE'].rolling(20).corr(data['brent_CLOSE'])
data['coal_Corr'] = data['TTF_CLOSE'].rolling(20).corr(data['coal_CLOSE'])
data['CO2_Corr'] = data['TTF_CLOSE'].rolling(20).corr(data['CO2_CLOSE'])
data = data.dropna()
X =  data[['TTF_RSI','brent_RSI','coal_RSI','CO2_RSI','brent_Corr','coal_Corr','CO2_Corr']].values
# data = pd.merge(data, pd.DataFrame(TTF['TTF_CLOSE']), how = 'inner', left_index=True, right_index=True)
display(data)

# train model with optimal number of hidden states

In [ ]:
X =  data.values
model = hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=1000).fit(X)
hidden_states = model.predict(X)
########################################################################### HMM #####################################
data['HMM_TTF'] = hidden_states
print(model.converged_)
display(data)

from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(model, 'HMM.pkl') 
# and later you can load it
model = joblib.load('HMM.pkl')
print(hidden_states)

In [ ]:
style_kwds = {'xtick.major.size': 1, 'ytick.major.size': 1,
              'font.family':u'courier prime code', 'legend.frameon': True}
sns.set(font_scale=1.5)

states = (pd.DataFrame(data, columns=['states'],index=data.index).join(data, how='inner')
          .reset_index(drop=False)
          )
# print(data.index)
# display(states)
sns.set_style('white', style_kwds)
# order = [0,1]
order = range(3)
fg = sns.FacetGrid(data=states, hue='HMM_TTF', hue_order=order,
                    aspect=1.31, size=12)
fg.map(plt.scatter, 'Date', 'TTF_CLOSE', alpha=0.8).add_legend()
sns.despine(offset=10)
fg.fig.suptitle('Historical WTI Regimes', fontsize=24, fontweight='demi')
fg.savefig('Hidden Markov (Mixture) Model_SPY Regimes.png')